### Transformar todos los chuncks de participant logs en un gran archivo parquet:

In [1]:
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.compute as pc

In [2]:
import os

def get_files(path, extension):
    for file in os.listdir(path):
        if os.path.isfile(os.path.join(path, file)) and file.endswith( extension ):
            yield file

In [3]:
schema = pa.schema([
    pa.field('timestamp', pa.string()),
    pa.field('currentLocation', pa.string()),
    pa.field('participantId', pa.int64()),
    pa.field('currentMode', pa.string()),
    pa.field('hungerStatus',pa.string()),
    pa.field('sleepStatus', pa.string()),
    pa.field('apartmentId', pa.int64()),
    pa.field('availableBalance', pa.float64()),
    pa.field('jobId', pa.int64()),
    pa.field('financialStatus', pa.string()),
    pa.field('dailyFoodBudget', pa.int64()),
    pa.field('weeklyExtraBudget', pa.float64()),
])

tables = []
folder = './Datasets/Activity Logs/'
for file in get_files(folder, '.parquet'):
    print('Procesando el archivo ' + str(file))
    t = pq.read_table(os.path.join(folder, file), schema=schema)
    tables.append(t)

Procesando el archivo ParticipantStatusLogs1.parquet
Procesando el archivo ParticipantStatusLogs10.parquet
Procesando el archivo ParticipantStatusLogs11.parquet
Procesando el archivo ParticipantStatusLogs12.parquet
Procesando el archivo ParticipantStatusLogs13.parquet
Procesando el archivo ParticipantStatusLogs14.parquet
Procesando el archivo ParticipantStatusLogs15.parquet
Procesando el archivo ParticipantStatusLogs16.parquet
Procesando el archivo ParticipantStatusLogs17.parquet
Procesando el archivo ParticipantStatusLogs18.parquet
Procesando el archivo ParticipantStatusLogs19.parquet
Procesando el archivo ParticipantStatusLogs2.parquet
Procesando el archivo ParticipantStatusLogs20.parquet
Procesando el archivo ParticipantStatusLogs21.parquet
Procesando el archivo ParticipantStatusLogs22.parquet
Procesando el archivo ParticipantStatusLogs23.parquet
Procesando el archivo ParticipantStatusLogs24.parquet
Procesando el archivo ParticipantStatusLogs25.parquet
Procesando el archivo Particip

In [4]:
final_table = pa.concat_tables(tables)
final_table

pyarrow.Table
timestamp: string
currentLocation: string
participantId: int64
currentMode: string
hungerStatus: string
sleepStatus: string
apartmentId: int64
availableBalance: double
jobId: int64
financialStatus: string
dailyFoodBudget: int64
weeklyExtraBudget: double
----
timestamp: [["2022-03-01T00:00:00Z","2022-03-01T00:00:00Z","2022-03-01T00:00:00Z","2022-03-01T00:00:00Z","2022-03-01T00:00:00Z",...,"2022-03-01T10:45:00Z","2022-03-01T10:45:00Z","2022-03-01T10:45:00Z","2022-03-01T10:45:00Z","2022-03-01T10:45:00Z"],["2022-03-01T10:45:00Z","2022-03-01T10:45:00Z","2022-03-01T10:45:00Z","2022-03-01T10:45:00Z","2022-03-01T10:45:00Z",...,"2022-03-01T21:35:00Z","2022-03-01T21:35:00Z","2022-03-01T21:35:00Z","2022-03-01T21:35:00Z","2022-03-01T21:35:00Z"],...,["2022-04-25T09:30:00Z","2022-04-25T09:30:00Z","2022-04-25T09:30:00Z","2022-04-25T09:30:00Z","2022-04-25T09:30:00Z",...,"2022-04-25T21:55:00Z","2022-04-25T21:55:00Z","2022-04-25T21:55:00Z","2022-04-25T21:55:00Z","2022-04-25T21:55:00Z"],["2

In [5]:
arr = pc.replace_substring(final_table["timestamp"], "T", " ")
arr = pc.replace_substring(arr, "Z", "")
arr

[
  [
    "2022-03-01 00:00:00",
    "2022-03-01 00:00:00",
    "2022-03-01 00:00:00",
    "2022-03-01 00:00:00",
    "2022-03-01 00:00:00",
    ...
    "2022-03-01 10:45:00",
    "2022-03-01 10:45:00",
    "2022-03-01 10:45:00",
    "2022-03-01 10:45:00",
    "2022-03-01 10:45:00"
  ],
  [
    "2022-03-01 10:45:00",
    "2022-03-01 10:45:00",
    "2022-03-01 10:45:00",
    "2022-03-01 10:45:00",
    "2022-03-01 10:45:00",
    ...
    "2022-03-01 21:35:00",
    "2022-03-01 21:35:00",
    "2022-03-01 21:35:00",
    "2022-03-01 21:35:00",
    "2022-03-01 21:35:00"
  ],
...,
  [
    "2022-04-25 09:30:00",
    "2022-04-25 09:30:00",
    "2022-04-25 09:30:00",
    "2022-04-25 09:30:00",
    "2022-04-25 09:30:00",
    ...
    "2022-04-25 21:55:00",
    "2022-04-25 21:55:00",
    "2022-04-25 21:55:00",
    "2022-04-25 21:55:00",
    "2022-04-25 21:55:00"
  ],
  [
    "2022-04-25 21:55:00",
    "2022-04-25 21:55:00",
    "2022-04-25 21:55:00",
    "2022-04-25 21:55:00",
    "2022-04-25 21:55:0

In [6]:
final_table.set_column(0, "timestamp", pc.cast(arr, pa.timestamp("ms")))

pyarrow.Table
timestamp: timestamp[ms]
currentLocation: string
participantId: int64
currentMode: string
hungerStatus: string
sleepStatus: string
apartmentId: int64
availableBalance: double
jobId: int64
financialStatus: string
dailyFoodBudget: int64
weeklyExtraBudget: double
----
timestamp: [[2022-03-01 00:00:00.000,2022-03-01 00:00:00.000,2022-03-01 00:00:00.000,2022-03-01 00:00:00.000,2022-03-01 00:00:00.000,...,2022-04-26 00:25:00.000,2022-04-26 00:25:00.000,2022-04-26 00:25:00.000,2022-04-26 00:25:00.000,2022-04-26 00:25:00.000]]
currentLocation: [["POINT (-2724.6277665310454 6866.2081834436985)","POINT (-1526.9372331431534 5582.2951345645315)","POINT (-1360.9905987829304 2108.804385379679)","POINT (-1558.517200825967 5600.664347152427)","POINT (976.2409614204214 4574.575079082071)",...,"POINT (-782.8905310061416 3992.5134090611436)","POINT (-3589.119754704547 6809.829648128866)","POINT (-3389.0663059732633 6103.41045979951)","POINT (-3243.993269315782 6127.8699068668675)","POINT (-

In [7]:
pq.write_table(final_table, "./participant_logs.parquet")

### Procesar participant logs para preparar para clustering (grouping por participant id, pivot por mes):

In [112]:
import pandas as pd

In [113]:
part_sum = pd.read_csv('participant_logs_summary.csv')

In [114]:
part_sum.head()

,month,participantId,apartmentId,jobId,monthlyAvgBalance,monthlyFoodBudget,monthlyExtraBudget
0,2022-03-01,0.0,926.0,254.0,5647.804675,360.0,10571.387363
1,2022-03-01,1.0,928.0,929.0,4255.881651,360.0,7560.279658
2,2022-03-01,2.0,291.0,348.0,4515.623553,480.0,8076.415386
3,2022-03-01,3.0,1243.0,316.0,4266.725347,360.0,6680.278156
4,2022-03-01,4.0,375.0,332.0,3514.953610,600.0,14107.025379


In [115]:
part_sum = part_sum.pivot(index="participantId", columns="month", values="monthlyAvgBalance").dropna()

In [116]:
part_sum.to_csv("participant_clustering.csv")

### Preprocessing de jobs:

In [117]:
jobs = pd.read_csv('./Datasets/Attributes/Jobs.csv')

In [118]:
jobs.head()

,jobId,employerId,hourlyRate,startTime,endTime,daysToWork,educationRequirement
0,0,379,10.000000,7:46:00 AM,3:46:00 PM,"[Monday,Tuesday,Wednesday,Thursday,Friday]",HighSchoolOrCollege
1,1,379,22.217633,7:31:00 AM,3:31:00 PM,"[Monday,Tuesday,Wednesday,Thursday,Friday]",Bachelors
2,2,380,10.000000,8:00:00 AM,4:00:00 PM,"[Monday,Tuesday,Wednesday,Thursday,Friday]",Bachelors
3,3,380,15.312071,7:39:00 AM,3:39:00 PM,"[Monday,Tuesday,Wednesday,Thursday,Friday]",Bachelors
4,4,381,21.355409,7:53:00 AM,3:53:00 PM,"[Monday,Tuesday,Wednesday,Thursday,Friday]",HighSchoolOrCollege


In [119]:
jobs["numberOfDays"] = len(jobs["daysToWork"].str.split(',',expand=True).columns)

In [120]:
jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1328 entries, 0 to 1327
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   jobId                 1328 non-null   int64  
 1   employerId            1328 non-null   int64  
 2   hourlyRate            1328 non-null   float64
 3   startTime             1328 non-null   object 
 4   endTime               1328 non-null   object 
 5   daysToWork            1328 non-null   object 
 6   educationRequirement  1328 non-null   object 
 7   numberOfDays          1328 non-null   int64  
dtypes: float64(1), int64(3), object(4)
memory usage: 83.1+ KB


In [121]:
from datetime import datetime, date

In [122]:
jobs["timeDeltas"] = jobs["endTime"].apply(lambda t: datetime.strptime(t, "%I:%M:%S %p")) - jobs["startTime"].apply(lambda t: datetime.strptime(t, "%I:%M:%S %p"))

In [123]:
jobs["hoursPerDay"] = jobs["timeDeltas"].apply(lambda t: t.seconds / 3600)

In [124]:
jobs["monthlySalary"] = jobs["hoursPerDay"] * jobs["numberOfDays"] * jobs["hourlyRate"] * 4

In [125]:
jobs[["jobId", "monthlySalary", "educationRequirement"]].set_index("jobId").to_csv("job_summary.csv")

### Combinación de tablas para obtener la tabla final con la historia mensual de cada participante:

In [66]:
part_clusters = pd.read_csv('participant_clusters.csv')
part_clusters.head()

,month,participantId,apartmentId,jobId,monthlyAvgBalance,monthlyFoodBudget,monthlyExtraBudget,Cluster
0,2022-03-01 00:00:00,0.0,926.0,254.0,5647.804675,360.0,10571.387363,C5
1,2022-03-01 00:00:00,1.0,928.0,929.0,4255.881651,360.0,7560.279658,C5
2,2022-03-01 00:00:00,2.0,291.0,348.0,4515.623553,480.0,8076.415386,C5
3,2022-03-01 00:00:00,3.0,1243.0,316.0,4266.725347,360.0,6680.278156,C5
4,2022-03-01 00:00:00,4.0,375.0,332.0,3514.953610,600.0,14107.025379,C5


In [67]:
jobs = pd.read_csv('job_summary.csv')
jobs.head()

,jobId,monthlySalary,educationRequirement
0,0,1600.000000,HighSchoolOrCollege
1,1,3554.821338,Bachelors
2,2,1600.000000,Bachelors
3,3,2449.931302,Bachelors
4,4,3416.865486,HighSchoolOrCollege


In [69]:
apartments = pd.read_csv('./Datasets/Attributes/Apartments.csv')
apartments.head()

,apartmentId,rentalCost,maxOccupancy,numberOfRooms,location,buildingId
0,1,768.16,2,4,POINT (1077.6979444315298 648.4427163702453),340
1,2,1014.55,2,1,POINT (-185.9292838076562 1520.3270983045118),752
2,3,1057.39,4,3,POINT (2123.0141855392585 5126.753457243003),639
3,4,1259.10,4,3,POINT (2103.6301776944765 4266.932930123476),397
4,5,411.50,1,4,POINT (7.0589743819342985 79.96163671849988),628


In [70]:
apartments = apartments[["apartmentId", "rentalCost"]]
apartments

,apartmentId,rentalCost
0,1,768.16
1,2,1014.55
2,3,1057.39
3,4,1259.10
4,5,411.50
...,...,...
1512,1729,703.80
1513,1730,1104.62
1514,1731,890.69
1515,1732,703.80


In [84]:
final_table = part_clusters.join(jobs, on='jobId', rsuffix='_2').join(apartments, on='apartmentId', rsuffix='_2')[["month", "participantId", "monthlyAvgBalance", "monthlyFoodBudget", "monthlyExtraBudget", "monthlySalary", "educationRequirement", "rentalCost", "Cluster"]].set_index("month")
final_table = final_table.fillna(final_table.mean())
final_table.head()

,participantId,monthlyAvgBalance,monthlyFoodBudget,monthlyExtraBudget,monthlySalary,educationRequirement,rentalCost,Cluster
month,,,,,,,,
2022-03-01 00:00:00,0.0,5647.804675,360.0,10571.387363,8225.600056,HighSchoolOrCollege,965.66,C5
2022-03-01 00:00:00,1.0,4255.881651,360.0,7560.279658,7228.227325,HighSchoolOrCollege,615.60,C5
2022-03-01 00:00:00,2.0,4515.623553,480.0,8076.415386,6334.912315,HighSchoolOrCollege,974.58,C5
2022-03-01 00:00:00,3.0,4266.725347,360.0,6680.278156,6122.812550,HighSchoolOrCollege,1100.37,C5
2022-03-01 00:00:00,4.0,3514.953610,600.0,14107.025379,3046.987928,Bachelors,872.51,C5


In [86]:
final_table.to_csv("participants_monthly_history.csv")

### Estadísticas por cluster y totales:

In [89]:
part_history = pd.read_csv("participants_monthly_history.csv")
part_history.head()

,month,participantId,monthlyAvgBalance,monthlyFoodBudget,monthlyExtraBudget,monthlySalary,educationRequirement,rentalCost,Cluster
0,2022-03-01 00:00:00,0.0,5647.804675,360.0,10571.387363,8225.600056,HighSchoolOrCollege,965.66,C5
1,2022-03-01 00:00:00,1.0,4255.881651,360.0,7560.279658,7228.227325,HighSchoolOrCollege,615.60,C5
2,2022-03-01 00:00:00,2.0,4515.623553,480.0,8076.415386,6334.912315,HighSchoolOrCollege,974.58,C5
3,2022-03-01 00:00:00,3.0,4266.725347,360.0,6680.278156,6122.812550,HighSchoolOrCollege,1100.37,C5
4,2022-03-01 00:00:00,4.0,3514.953610,600.0,14107.025379,3046.987928,Bachelors,872.51,C5


In [100]:
monthly_summary = part_history.groupby(["month"]).mean().drop(columns=["participantId"])
monthly_summary.head()

,monthlyAvgBalance,monthlyFoodBudget,monthlyExtraBudget,monthlySalary,rentalCost
month,,,,,
2022-03-01 00:00:00,2786.669244,478.100890,4354.482642,3470.226913,829.401632
2022-04-01 00:00:00,5382.469875,467.727273,7645.336970,3730.115215,822.273949
2022-05-01 00:00:00,8044.968700,467.727273,10363.401434,3731.037621,828.073242
2022-06-01 00:00:00,10757.997664,467.727273,13593.676851,3731.037621,826.061858
2022-07-01 00:00:00,13475.350406,467.727273,16194.288448,3731.037621,833.217270


In [101]:
monthly_summary.to_csv("monthly_summary.csv")

In [110]:
cluster_history = part_history.groupby(["month", "Cluster"]).mean().reset_index().set_index("month").drop(columns=["participantId"])
monthly_summary["Cluster"] = "Todos"
cluster_history = pd.concat([cluster_history, monthly_summary])
cluster_history.head(-10)

,Cluster,monthlyAvgBalance,monthlyFoodBudget,monthlyExtraBudget,monthlySalary,rentalCost
month,,,,,,
2022-03-01 00:00:00,C1,2691.856263,454.100719,4572.825477,3270.288913,833.335529
2022-03-01 00:00:00,C2,8061.307717,380.000000,14355.460378,11132.021938,800.546601
2022-03-01 00:00:00,C3,1510.232456,513.292683,2615.870898,2151.544320,827.323699
2022-03-01 00:00:00,C4,3735.495296,442.247191,6813.303008,4771.284480,839.471813
2022-03-01 00:00:00,C5,5526.971095,407.272727,9540.894967,7093.766731,811.603457
...,...,...,...,...,...,...
2022-03-01 00:00:00,Todos,2786.669244,478.100890,4354.482642,3470.226913,829.401632
2022-04-01 00:00:00,Todos,5382.469875,467.727273,7645.336970,3730.115215,822.273949
2022-05-01 00:00:00,Todos,8044.968700,467.727273,10363.401434,3731.037621,828.073242


In [111]:
cluster_history.to_csv("cluster_history.csv")